In [ ]:
import cv2
import numpy as np

def preprocess_image(image):
    # Convert to grayscale if the image is not already
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Find the binary mask
    _, binary = cv2.threshold(image, 1, 255, cv2.THRESH_BINARY)
    return binary

def compute_centroid_and_orientation(image):
    # Compute the moments of the binary image
    M = cv2.moments(image)
    # Compute centroid
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    # Compute orientation
    mu20 = M['mu20'] / M['m00']
    mu02 = M['mu02'] / M['m00']
    mu11 = M['mu11'] / M['m00']
    theta = 0.5 * np.arctan2(2 * mu11, mu20 - mu02)
    return (cx, cy), theta

def align_images(img1, img2):
    # Preprocess images
    binary1 = preprocess_image(img1)
    binary2 = preprocess_image(img2)
    
    # Compute centroids and orientations
    (cx1, cy1), theta1 = compute_centroid_and_orientation(binary1)
    (cx2, cy2), theta2 = compute_centroid_and_orientation(binary2)
    
    # Compute the translation needed to align the centroids
    translation = (cx2 - cx1, cy2 - cy1)
    
    # Compute the rotation needed to align the orientations
    rotation = np.degrees(theta2 - theta1)
    
    # Create transformation matrix
    M_translation = np.float32([[1, 0, translation[0]], [0, 1, translation[1]]])
    rows, cols = img1.shape[:2]
    img1_translated = cv2.warpAffine(img1, M_translation, (cols, rows))
    
    # Rotate the translated image
    M_rotation = cv2.getRotationMatrix2D((cx2, cy2), rotation, 1)
    img1_aligned = cv2.warpAffine(img1_translated, M_rotation, (cols, rows))
    
    return img1_aligned

# Load images
img1 = cv2.imread('image1.png')
img2 = cv2.imread('image2.png')

# Align images
img1_aligned = align_images(img1, img2)

# Display results
cv2.imshow('Image 1 Aligned', img1_aligned)
cv2.imshow('Image 2', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()
